<a href="https://colab.research.google.com/github/Vargol/StableDiffusionColabs/blob/main/SDXL/stable_diffusion_sdxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy

In [ ]:
import mediapy as media
import random
import sys
import torch
import gc
import time

from diffusers import DiffusionPipeline, AutoencoderKL, UniPCMultistepScheduler

refiner = None
pipe = None

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",
                                    torch_dtype=torch.float16,
                                    force_upcast=False).to('cuda')


styles = {
"Enhance" : {
"Positive" : "breathtaking {prompt} . award-winning, professional, highly detailed",
"Negative" : "ugly, deformed, noisy, blurry, distorted, grainy",
},
"Anime" : {
"Positive" : "anime artwork {prompt} . anime style, key visual, vibrant, studio anime,  highly detailed",
"Negative" : "photo, deformed, black and white, realism, disfigured, low contrast",
},
"Photographic" : {
"Positive" : "cinematic photo {prompt} . 35mm photograph, film, bokeh, professional, 4k, highly detailed",
"Negative" : "drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly",
},
"Digital art" : {
"Positive" : "concept art {prompt} . digital artwork, illustrative, painterly, matte painting, highly detailed",
"Negative" : "photo, photorealistic, realism, ugly",
},
"Comic book" : {
"Positive" : "comic {prompt} . graphic illustration, comic art, graphic novel art, vibrant, highly detailed",
"Negative" : "photograph, deformed, glitch, noisy, realistic, stock photo",
},
"Fantasy art" : {
"Positive" : "ethereal fantasy concept art of  {prompt} . magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy",
"Negative" : "photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white",
},
"Analog film" : {
"Positive" : "analog film photo {prompt} . faded film, desaturated, 35mm photo, grainy, vignette, vintage, Kodachrome, Lomography, stained, highly detailed, found footage",
"Negative" : "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured",
},
"Neonpunk" : {
"Positive" : "neonpunk style {prompt} . cyberpunk, vaporwave, neon, vibes, vibrant, stunningly beautiful, crisp, detailed, sleek, ultramodern, magenta highlights, dark purple shadows, high contrast, cinematic, ultra detailed, intricate, professional",
"Negative" : "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured",
},
"Isometric" : {
"Positive" : "isometric style {prompt} . vibrant, beautiful, crisp, detailed, ultra detailed, intricate",
"Negative" : "deformed, mutated, ugly, disfigured, blur, blurry, noise, noisy, realistic, photographic",
},
"Lowpoly" : {
"Positive" : "low-poly style {prompt} . low-poly game art, polygon mesh, jagged, blocky, wireframe edges, centered composition",
"Negative" : "noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo",
},
"Origami" : {
"Positive" : "origami style {prompt} . paper art, pleated paper, folded, origami art, pleats, cut and fold, centered composition",
"Negative" : "noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo",
},
"Line art" : {
"Positive" : "line art drawing {prompt} . professional, sleek, modern, minimalist, graphic, line art, vector graphics",
"Negative" : "anime, photorealistic, 35mm film, deformed, glitch, blurry, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, mutated, realism, realistic, impressionism, expressionism, oil, acrylic",
},
"Craft clay" : {
"Positive" : "play-doh style {prompt} . sculpture, clay art, centered composition, Claymation",
"Negative" : "sloppy, messy, grainy, highly detailed, ultra textured, photo",
},
"Cinematic" : {
"Positive" : "cinematic film still {prompt} . shallow depth of field, vignette, highly detailed, high budget Hollywood movie, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy",
"Negative" : "anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured",
},
"3d-model" : {
"Positive" : "professional 3d model {prompt} . octane render, highly detailed, volumetric, dramatic lighting",
"Negative" : "ugly, deformed, noisy, low poly, blurry, painting",
},
"Pixel art" : {
"Positive" : "pixel-art {prompt} . low-res, blocky, pixel art style, 8-bit graphics",
"Negative" : "sloppy, messy, blurry, noisy, highly detailed, ultra textured, photo, realistic",
},
"Texture" : {
"Positive" : "texture {prompt} top down close-up",
"Negative" : "ugly, deformed, noisy, blurry",
}
}

aspects = {
"12:5" :  { 'x' : 1536  , 'y' :  640 },
"7:4" :   { 'x' : 1344  , 'y' :  768 },
"19:13" : { 'x' : 1216  , 'y' :  832  },
"9:7" :   { 'x' : 1152 , 'y' : 896  },
"1:1" :   { 'x' : 1024 , 'y' : 1024 },
"7:9" :   { 'x' : 896  , 'y' : 1152  },
"13:19" : { 'x' : 832  , 'y' :  1216 },
"4:7" :   { 'x' : 768  , 'y' :  1344 },
"5:12" :  { 'x' : 640  , 'y' :  1536 }
}




The next cell is where you set you parameters and run Stable Diffusion.
The cell is re-runable, so you can edit the parameters and run without recreating the environment

Note the free tier can't hold the base and refiner models in VRAM at the same time so if the refiner is userd it does the base run image_count times followed by the refiner run image_count times, it uses a seperate generator for each run of the base model so they can be passed to the refiner and allow you to regenerate an image using the prompt, style and seed (requires code changes).

In [ ]:
prompt = 'A red haired woman standing in a lush green jungle'

image_count = 4
use_refiner = True
base_refiner_split=0.8
num_inference_steps=45
style = "Photographic"
aspect = "1:1"

orig_prompt = prompt
prompt = styles[style]["Positive"].replace("{prompt}", prompt)
negative_prompt = styles[style]["Negative"]

torch.cuda.empty_cache()
base_output = []
prompt_details = []
generators = []


if pipe is None:

  refiner = None
  gc.collect()

  pipe = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-base-1.0",
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
      vae=vae
      ).to('cuda')

  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
  pipe.enable_vae_tiling()

base_output = []
prompt_details = []

for i in range(image_count):

  seed = random.randint(0, sys.maxsize)
  generator =  torch.Generator("cuda").manual_seed(seed);

  kwargs = {
      'prompt':prompt,
      'negative_prompt': negative_prompt,
      'output_type': "latent" if use_refiner else "pil",
      'generator':generator,
      'num_inference_steps':num_inference_steps,
      'width': aspects[aspect]['x'],
      'height': aspects[aspect]['y'],
  }

  if use_refiner:
    kwargs['denoising_end'] = base_refiner_split

  images = pipe(**kwargs).images

  if use_refiner:
    generators.append(generator)
    base_output.append(images)
    prompt_details.append(f"Prompt:\t{orig_prompt}\nStyle:\t{style}\nSeed:\t{seed}")
  else:
    print(f"Prompt:\t{orig_prompt}\nStyle:\t{style}\nSeed:\t{seed}")
    media.show_images(images)

if use_refiner:

  pipe = None
  gc.collect()
  torch.cuda.empty_cache()

  refiner = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      vae=vae,
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
  ).to('cuda')

  refiner.scheduler = UniPCMultistepScheduler.from_config(refiner.scheduler.config)


  refiner.enable_vae_slicing()
  refiner.enable_vae_tiling()

  gc.collect()

  for i in range(image_count):

    if use_refiner:
      images = refiner(
          prompt = prompt,
          negative_prompt=negative_prompt,
          image = base_output[i],
          num_inference_steps=num_inference_steps,
          denoising_start = base_refiner_split,
          generator=generators[i],
          ).images

    print(prompt_details[i])
    media.show_images(images)
